In [3]:
from utils_eval import compute_pairwise_metrics, extract_score
import json, os, numpy as np, pandas as pd
from IPython.display import display

def evaluate_models(eval_fn):

    with open(eval_fn) as f:
        data = json.load(f)

    id2data = {d["id"]: d for d in data}

    models = set([])
    for fn in os.listdir("data/preds"):
        with open(f"data/preds/{fn}") as f:
            model_name = fn.replace(".jsonl", "").replace("preds_", "")
            # if "gem-1p5" in model_name:
            #     continue
            models.add(model_name)
            for line in f:
                d = json.loads(line)
                if d["input_fn"] != eval_fn:
                    continue
                id2data[d["id"]]["pred_" + model_name] = d["output"]

    sample_types = {}
    for d in data:
        if d["sample_type"] not in sample_types:
            sample_types[d["sample_type"]] = []
        sample_types[d["sample_type"]].append(d)

    results, N_samples = [], []
    total_N_samples = {"model": "Total"}
    for sample_type in sorted(sample_types):
        total_N_samples[sample_type] = len(sample_types[sample_type])
    N_samples.append(total_N_samples)

    # remove models that don't have any samples annotated
    models = [model for model in models if any("pred_" + model in d for d in data)]

    print(models)

    bad_ends = ["-b", "-c", "-d", "-e"]
    for model in models:
        # if any(bad_end in model for bad_end in bad_ends):
        #     continue
        N_samples_row = {"model": model}
        result_row = {"model": model}
        for sample_type in sorted(sample_types):
            model_samples = [d for d in sample_types[sample_type] if "pred_" + model in d]
            if len(model_samples) == 0:
                continue
            N_samples_row[sample_type] = len(model_samples)
            if sample_type.startswith("pairwise"):
                pref1, acc, err = compute_pairwise_metrics(model_samples, model)
                result_row[sample_type] = acc
            else:
                y_true = [d["zscore"] for d in model_samples]
                y_pred = []
                for d in model_samples:
                    pred, err = extract_score(d, "pred_" + model)
                    y_pred.append(pred)
                abs_err = np.abs(np.array(y_true) - np.array(y_pred))
                corr = np.corrcoef(y_true, y_pred)[0, 1]
                avg_R = np.mean(y_pred)
                result_row[sample_type+"_MAE_R"] = abs_err.mean()
                result_row[sample_type+"_Corr_R"] = corr
                result_row[sample_type+"_Avg_R"] = avg_R
        N_samples.append(N_samples_row)

        results.append(result_row)

    # add the model eval_fn as header to the results
    print(eval_fn.center(80, "-"))
    display(pd.DataFrame(results).sort_values(by="pairwise", ascending=False).set_index("model").round(2))
    display(pd.DataFrame(N_samples).set_index("model").round(2))

evaluate_models(eval_fn="data/lamp_PRGS_test.json")
evaluate_models(eval_fn="data/lamp_PR_editor_test.json")

['lamp-gem-1p5-flash-s-a', 'lamp-gem-1p5-flash-p-c', 'lamp-gem-1p5-flash-p-b', 'gpt-4o-2024-08-06', 'lamp-gem-1p5-flash-ps-b', 'lamp-gem-1p5-flash-ps-a', 'lamp-gem-1p5-flash-p-e', 'lamp-gem-1p5-flash-p-a', 'lamp-gem-1p5-flash-p-d', 'llama_3.2_1b_results_constant', 'gpt-4o-mini', 'llama_3.2_1b_results_cosine', 'lamp-gem-1p5-flash-s-b', 'gemini-1.5-flash', 'lamp-4o-p', 'lamp-4o-mini-p', 'llama_3.2_1b_results_parallel']
----------------------------data/lamp_PRGS_test.json----------------------------


,pairwise,pairwise-P1,pairwise-P2,pairwise-P3,pairwise-P4,pairwise-P5,pairwise-P6,pairwise-P7,pairwise-gold,pairwise-silver,reward_MAE_R,reward_Corr_R,reward_Avg_R
model,,,,,,,,,,,,,
lamp-4o-p,100.00,92.09,96.74,98.56,100.00,100.00,100.00,99.28,73.55,99.82,1.50,0.38,5.74
lamp-gem-1p5-flash-ps-b,100.00,87.44,96.74,98.09,99.50,99.45,100.00,100.00,73.96,100.00,2.01,0.40,5.78
lamp-gem-1p5-flash-ps-a,100.00,84.65,95.81,98.56,99.50,100.00,100.00,100.00,73.88,100.00,2.80,0.43,7.54
lamp-4o-mini-p,99.01,88.84,93.49,95.69,97.99,98.91,100.00,99.28,72.72,99.91,3.27,0.28,2.51
llama_3.2_1b_results_parallel,92.57,77.21,76.28,80.86,80.90,90.16,91.19,93.48,69.15,98.39,1.34,0.43,5.05
llama_3.2_1b_results_cosine,89.60,71.63,76.28,77.99,78.89,84.70,93.08,86.96,69.98,98.75,1.40,0.42,5.27
llama_3.2_1b_results_constant,86.14,70.70,77.21,75.12,76.38,81.42,84.28,82.61,65.34,95.71,1.38,0.37,4.89
lamp-gem-1p5-flash-s-a,80.20,55.35,57.21,63.16,63.32,71.04,69.81,73.91,66.09,100.00,3.17,0.19,8.09
lamp-gem-1p5-flash-s-b,79.46,53.02,54.88,66.51,64.82,68.85,71.07,72.46,65.34,100.00,3.21,0.16,8.14


,pairwise,pairwise-P1,pairwise-P2,pairwise-P3,pairwise-P4,pairwise-P5,pairwise-P6,pairwise-P7,pairwise-gold,pairwise-silver,reward
model,,,,,,,,,,,
Total,404,215,215,209,199,183,159,138,1206,1120,430
lamp-gem-1p5-flash-s-a,404,215,215,209,199,183,159,138,1206,1120,430
lamp-gem-1p5-flash-p-c,404,215,215,209,199,183,159,138,1206,1120,430
lamp-gem-1p5-flash-p-b,404,215,215,209,199,183,159,138,1206,1120,430
gpt-4o-2024-08-06,404,215,215,209,199,183,159,138,1206,1120,430
lamp-gem-1p5-flash-ps-b,404,215,215,209,199,183,159,138,1206,1120,430
lamp-gem-1p5-flash-ps-a,404,215,215,209,199,183,159,138,1206,1120,430
lamp-gem-1p5-flash-p-e,404,215,215,209,199,183,159,138,1206,1120,430
lamp-gem-1p5-flash-p-a,404,215,215,209,199,183,159,138,1206,1120,430


['gpt-4o-mini-2024-07-18', 'lamp-4o-mini-p-editor']
-------------------------data/lamp_PR_editor_test.json--------------------------


,pairwise,pairwise-P1,pairwise-P2,pairwise-P3,pairwise-P4,pairwise-P5,pairwise-P6,pairwise-P7,reward_MAE_R,reward_Corr_R,reward_Avg_R
model,,,,,,,,,,,
lamp-4o-mini-p-editor,98.64,84.47,92.83,97.16,98.80,98.83,99.60,99.46,3.61,-0.42,3.90
gpt-4o-mini-2024-07-18,9.43,32.62,18.44,11.60,6.01,4.69,4.38,3.76,3.26,-0.07,8.48


,pairwise,pairwise-P1,pairwise-P2,pairwise-P3,pairwise-P4,pairwise-P5,pairwise-P6,pairwise-P7,reward
model,,,,,,,,,
Total,880,515,488,457,416,341,251,186,1030
gpt-4o-mini-2024-07-18,880,515,488,457,416,341,251,186,1030
lamp-4o-mini-p-editor,880,515,488,457,416,341,251,186,1030
